# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [50]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re
# NLP libraries
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
#ML Libraries
from sklearn.pipeline import Pipeline
#from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
#Warnings
import warnings
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gustavo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Gustavo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
# load data from database
def load_data():
    '''Function that loads that data from a SQL database and 
    returns the feature X and target variables Y''' 
    warnings.simplefilter('ignore')
    engine = create_engine('sqlite:///DisasterMessages.db')
    df = pd.read_sql("SELECT * FROM Messages",engine)
    #X = df.iloc[:,1]
    #Y = df.iloc[:,4:]
    #Only for testing
    X = df.iloc[:100,1]
    Y = df.iloc[:100,4:]
    return X,Y

#### Some testing to help define the function load data

In [3]:
engine = create_engine('sqlite:///DisasterMessages.db')
df = pd.read_sql("SELECT * FROM Messages",engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X0 = df.iloc[:10,1]
X0

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
5               Information about the National Palace-
6                       Storm at sacred heart of jesus
7    Please, we need tents and water. We are in Sil...
8      I would like to receive the messages, thank you
9    I am in Croix-des-Bouquets. We have health iss...
Name: message, dtype: object

In [5]:
X1 = df.message.values
X1[:5]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight'],
      dtype=object)

In [6]:
df.iloc[:10,4:].head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [29]:
def tokenize(text):
    warnings.simplefilter('ignore')    
    # Text Normalization(remove any characters that are not letters or numbers)
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize text
    tokens = word_tokenize(text)
    #remove stop words // Note: sometimes removing all the stopwords results in just one 
    #single word without a concrete meaning 
    
    #tokens = [w for w in tokens if w not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [30]:
#Checking the first two functions
X,y = load_data()
for message in X:
    tokens = tokenize(message)
    print(message)
    print(tokens,'\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 

Information about the National Palace-
['information', 'about', 'the', 'national', 'palace'] 

Storm at sacred heart of jesus
['storm', 'at', 'sacred', 'heart', 'of', 'jesus'] 

Please, we need tents and water. We are in Silo, Thank

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### Additional References
* [Udacity Knowledge](https://knowledge.udacity.com/questions/52835)
* [Using GridSearch with MutiOutputClassification](https://knowledge.udacity.com/questions/45533)
* [Solving Multi-Label Classification Problems] (https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/)
* [sklearn.multiclass.OneVsRestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html)


In [31]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier())))
    ])    

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [32]:
warnings.simplefilter('ignore')
#Loading data
X, y = load_data()
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)
# train classifier
pipeline.fit(X_train,y_train)
# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
# Only works for single classification 
def display_results(y_test, y_pred):
    #Ignore warning messages
    warnings.simplefilter('ignore')
    
    labels = np.unique(y_pred)
    class_rep = classification_report(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()
    
    print("Category: ",y_test.name)
    print("Labels:", labels)
    print("Classification Report:\n", class_rep)
    print("Accuracy:", accuracy)

In [12]:
y0_test = y_test.iloc[:,0]

In [13]:
y0_pred = y_pred[:,0]

In [14]:
y0_test.name

'related'

In [15]:
y_test.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [16]:
for c in range(len(y_test.columns)):
    ycol_test=y_test.iloc[:,c]
    print(ycol_test.name)

related
request
offer
aid_related
medical_help
medical_products
search_and_rescue
security
military
child_alone
water
food
shelter
clothing
money
missing_people
refugees
death
other_aid
infrastructure_related
transport
buildings
electricity
tools
hospitals
shops
aid_centers
other_infrastructure
weather_related
floods
storm
fire
earthquake
cold
other_weather
direct_report


In [17]:
#Showing results
display_results(y0_test, y0_pred)

Category:  related
Labels: [1.]
Classification Report:
               precision    recall  f1-score   support

         1.0       0.80      1.00      0.89        20

   micro avg       0.80      1.00      0.89        20
   macro avg       0.80      1.00      0.89        20
weighted avg       0.80      1.00      0.89        20

Accuracy: 0.8


In [34]:
#Multiclassification report 
def multiclass_report(y_test, y_pred):
    warnings.simplefilter('ignore')
    #iterating through the columns and calling sklearn's `classification_report` on each.
    for c in range(len(y_test.columns)):
        yc_test=y_test.iloc[:,c]
        yc_pred = y_pred[:,c]
           
        labels = np.unique(yc_pred)
        class_rep = classification_report(yc_test, yc_pred, labels=labels)
        accuracy = (yc_pred == yc_test).mean()
        print("----------------------------------------------------------------------")
        print("Category {}: {}".format(c+1,yc_test.name))
        print("Labels:", labels)
        print("Classification Report:\n", class_rep)
        print("Accuracy: {}\n".format(accuracy))
        print("----------------------------------------------------------------------")

In [35]:
#Showing Multioutput Classification Report
multiclass_report(y_test, y_pred)

----------------------------------------------------------------------
Category 1: related
Labels: [0. 1.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.20      0.33      0.25         3
         1.0       0.90      0.82      0.86        22

   micro avg       0.76      0.76      0.76        25
   macro avg       0.55      0.58      0.55        25
weighted avg       0.82      0.76      0.78        25

Accuracy: 0.76

----------------------------------------------------------------------
----------------------------------------------------------------------
Category 2: request
Labels: [0. 1.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.79      0.92      0.85        12
         1.0       0.91      0.77      0.83        13

   micro avg       0.84      0.84      0.84        25
   macro avg       0.85      0.84      0.84        25
weighted avg       0.85      0.84      0.84        2

----------------------------------------------------------------------
Category 29: weather_related
Labels: [0.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.84      1.00      0.91        21

   micro avg       0.84      1.00      0.91        21
   macro avg       0.84      1.00      0.91        21
weighted avg       0.84      1.00      0.91        21

Accuracy: 0.84

----------------------------------------------------------------------
----------------------------------------------------------------------
Category 30: floods
Labels: [0.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.92      1.00      0.96        23

   micro avg       0.92      1.00      0.96        23
   macro avg       0.92      1.00      0.96        23
weighted avg       0.92      1.00      0.96        23

Accuracy: 0.92

----------------------------------------------------------------------
---------------

In [36]:
#Function for steps 3 to 5 
def main_basic():
    warnings.simplefilter('ignore')
    #Loading data
    X, y = load_data()
    #Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # build pipeline
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier())))
     ])   
        
    # train classifier
    pipeline.fit(X_train,y_train)
    # predict on test data
    y_pred = pipeline.predict(X_test)
    # display results
    multiclass_report(y_test, y_pred)

In [37]:
#Running the model
main_basic()

----------------------------------------------------------------------
Category 1: related
Labels: [0. 1.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         6
         1.0       0.75      0.95      0.84        19

   micro avg       0.72      0.72      0.72        25
   macro avg       0.38      0.47      0.42        25
weighted avg       0.57      0.72      0.64        25

Accuracy: 0.72

----------------------------------------------------------------------
----------------------------------------------------------------------
Category 2: request
Labels: [0. 1.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.88      0.44      0.58        16
         1.0       0.47      0.89      0.62         9

   micro avg       0.60      0.60      0.60        25
   macro avg       0.67      0.66      0.60        25
weighted avg       0.73      0.60      0.59        2

Labels: [0.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.92      1.00      0.96        23

   micro avg       0.92      1.00      0.96        23
   macro avg       0.92      1.00      0.96        23
weighted avg       0.92      1.00      0.96        23

Accuracy: 0.92

----------------------------------------------------------------------
----------------------------------------------------------------------
Category 19: other_aid
Labels: [0.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.88      1.00      0.94        22

   micro avg       0.88      1.00      0.94        22
   macro avg       0.88      1.00      0.94        22
weighted avg       0.88      1.00      0.94        22

Accuracy: 0.88

----------------------------------------------------------------------
----------------------------------------------------------------------
Category 20: infrastructure_related
Label

----------------------------------------------------------------------
Category 35: other_weather
Labels: [0.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.96      1.00      0.98        24

   micro avg       0.96      1.00      0.98        24
   macro avg       0.96      1.00      0.98        24
weighted avg       0.96      1.00      0.98        24

Accuracy: 0.96

----------------------------------------------------------------------
----------------------------------------------------------------------
Category 36: direct_report
Labels: [0. 1.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.71      0.75      0.73        16
         1.0       0.50      0.44      0.47         9

   micro avg       0.64      0.64      0.64        25
   macro avg       0.60      0.60      0.60        25
weighted avg       0.63      0.64      0.63        25

Accuracy: 0.64

------------------------

### 6. Improve your model
Use grid search to find better parameters. 

In [38]:
pipeline.get_params(deep=True)

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000001249FF87730>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None

In [46]:
def build_model():
    '''Function that builds a ML Pipeline with optimization parameters '''
    #Ignore warning messages
    warnings.simplefilter('ignore')
    
    # build pipeline
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier())))
     ]) 
    # specify parameters for grid search
    parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__estimator__n_estimators': [10, 50, 100],
        'clf__estimator__estimator__min_samples_split': [2,3]
       
     }

    cv = GridSearchCV(pipeline, param_grid = parameters)
    
    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [47]:
def Opt_model():
    #Ignore warning messages
    warnings.simplefilter('ignore')
    
    #Loading data
    X, y = load_data()
    #Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    #Creating a ML Pipeline model with GridSearch Parameters
    model = build_model()
    # train classifier and find the best estimator
    model.fit(X_train, y_train)   
    #Getting the best estimator
    best_model = model.best_estimator_ 
    best_params = model.best_params_
    # predict on test data// GridSearchCV calls predict on the estimator with the best found parameters
    y_pred = model.predict(X_test)
    # display results
    print("Results for the Best Model with the Following Parameters\n{}\n".format(best_params))
    multiclass_report(y_test, y_pred)
    return best_model

In [48]:
best_model = Opt_model()

Results for the Best Model with the Following Parameters
{'clf__estimator__estimator__n_estimators': 10, 'tfidf__use_idf': True, 'vect__max_df': 1.0, 'vect__max_features': None, 'vect__ngram_range': (1, 1)}

----------------------------------------------------------------------
Category 1: related
Labels: [1.]
Classification Report:
               precision    recall  f1-score   support

         1.0       0.88      1.00      0.94        22

   micro avg       0.88      1.00      0.94        22
   macro avg       0.88      1.00      0.94        22
weighted avg       0.88      1.00      0.94        22

Accuracy: 0.88

----------------------------------------------------------------------
----------------------------------------------------------------------
Category 2: request
Labels: [0. 1.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.75      0.46      0.57        13
         1.0       0.59      0.83      0.69        12

   micro 

----------------------------------------------------------------------
Category 20: infrastructure_related
Labels: [0.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.92      1.00      0.96        23

   micro avg       0.92      1.00      0.96        23
   macro avg       0.92      1.00      0.96        23
weighted avg       0.92      1.00      0.96        23

Accuracy: 0.92

----------------------------------------------------------------------
----------------------------------------------------------------------
Category 21: transport
Labels: [0.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.96      1.00      0.98        24

   micro avg       0.96      1.00      0.98        24
   macro avg       0.96      1.00      0.98        24
weighted avg       0.96      1.00      0.98        24

Accuracy: 0.96

----------------------------------------------------------------------
-----

In [49]:
#Parameters of the optimized model
best_model.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...te=None, verbose=0,
            warm_start=False),
          n_jobs=None),
           n_jobs=None))])>

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Hyperparameter Optimization across multiple models
* [Hyperparameter optimization across multiple models in scikit-learn](http://www.davidsbatista.net/blog/2018/02/23/model_optimization/)
* [scikit grid search over multiple classifiers](https://stackoverflow.com/questions/23045318/scikit-grid-search-over-multiple-classifiers)
* [Multi-label Classification](https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff)
* **Check Project** `Finding_donors`


### 9. Export your model as a pickle file

In [54]:
# save the model to disk
filename = 'classifier.pkl'
pickle.dump(best_model, open(filename, 'wb'))

In [55]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [56]:
loaded_model.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...te=None, verbose=0,
            warm_start=False),
          n_jobs=None),
           n_jobs=None))])>

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.